---

Created for [learn-investments.rice-business.org](https://learn-investments.rice-business.org)
    
By [Kerry Back](https://kerryback.com) and [Kevin Crotty](https://kpcrotty.github.io/)
    
Jones Graduate School of Business, Rice University

---


# U.S. STOCK MARKET RETURN

## VOLATILITY

In [2]:
from pandas_datareader import DataReader as pdr

mkt = pdr('F-F_Research_Data_Factors_Daily','famafrench',start=1926)[0]/100
mkt = mkt['Mkt-RF'] + mkt.RF
mkt_vol = mkt.resample("M").apply(lambda x: x.std())[:-1]
mkt_vol.index = mkt_vol.index.to_period("M").astype(str)
mkt_vol.name = "market volatility"
mkt_vol.tail()


Date
2023-02    0.010208
2023-03    0.011875
2023-04    0.007787
2023-05    0.008283
2023-06    0.007137
Name: market volatility, dtype: float64

## MONTHLY RETURNS

In [3]:
mkt = pdr('F-F_Research_Data_Factors','famafrench',start=1926)[0]/100
mkt_ret = mkt['Mkt-RF'] + mkt.RF
mkt_ret.index = mkt_ret.index.astype(str)
mkt_ret.name = "market return"
mkt_ret.tail()

Date
2023-03    0.0287
2023-04    0.0096
2023-05    0.0071
2023-06    0.0686
2023-07    0.0366
Name: market return, dtype: float64

# RETURNS FROM YAHOO FINANCE

In [4]:
import yfinance as yf

ticker = input("Input a ticker")
price = yf.download(ticker, start="1970-01-01")["Adj Close"]

[*********************100%***********************]  1 of 1 completed


## VOLATILITY

In [5]:
ret_daily = price.pct_change()
ret_vol = ret_daily.resample("M").apply(lambda x: x.std())[:-1]
ret_vol.index = ret_vol.index.to_period("M").astype(str)
ret_vol.name = ticker + " volatility"
ret_vol.tail()

Date
2023-05    0.015990
2023-06    0.012575
2023-07    0.013905
2023-08    0.010208
2023-09    0.010815
Name: cvx volatility, dtype: float64

## MONTHLY RETURNS

In [6]:
price_monthly = price.resample("M").last()
ret_ret = price_monthly.pct_change()[:-1]
ret_ret.index = ret_ret.index.to_period("M").astype(str)
ret_ret.index.name = "month"
ret_ret.name = ticker + " return"
ret_ret.tail()

month
2023-05   -0.097788
2023-06    0.044682
2023-07    0.040102
2023-08   -0.006182
2023-09    0.046679
Name: cvx return, dtype: float64

# SELECT YAHOO RETURN OR MARKET

In [7]:
import pandas as pd

flag = int(input("input 0 for market or 1 for yahoo return"))
data = "ret" if flag else "mkt"
name = ticker.upper() if flag else "Market" 
vol = eval(data + "_vol")
ret = eval(data + "_ret")
df = pd.concat((vol, ret), axis=1).dropna()
df.columns = ["vol", "ret"]

# FIGURE 1

In [8]:
import plotly.graph_objects as go

trace = go.Scatter(
    x=vol.index.to_list(), 
    y=vol, 
    mode="lines", 
    hovertemplate="%{x}<br>%{y:.1%}<extra></extra>"
)
fig = go.Figure(trace)
fig.update_layout(
    yaxis_tickformat=".0%",
    yaxis_title = f"{name} Volatility",
    template="plotly_white",
)
fig.show()

# FIGURE 2

In [9]:
import plotly.express as px

fig = px.scatter(
    x=vol[:-1],
    y=vol[1:],
    trendline="ols",
)
fig.update_layout(
    xaxis_title=f"Lagged {name} Volatility",
    yaxis_title=f"{name} Volatility",
    xaxis_tickformat=".0%",
    yaxis_tickformat=".0%",
    template="plotly_white",
)
fig.update_traces(
    marker=dict(
        size=12, 
        line=dict(width=2, color="DarkSlateGrey")
    ),
    selector=dict(mode="markers"),
    text=df.index.to_list()[1:],
    hovertemplate="%{text}<br>lagged vol = %{x}<br>vol = %{y}<extra></extra>"
)
fig.show()


# FIGURE 3

In [10]:
fig = px.scatter(
    x=df.vol[:-1],
    y=df.ret[1:],
    trendline="ols",
)
fig.update_layout(
    xaxis_title=f"Lagged {name} Volatility",
    yaxis_title=f"{name} Return",
    xaxis_tickformat=".0%",
    yaxis_tickformat=".0%",
    template="plotly_white",
)
fig.update_traces(
    marker=dict(
        size=12, 
        line=dict(width=2, color="DarkSlateGrey")
    ),
    selector=dict(mode="markers"),
    text=df.index.to_list()[1:],
    hovertemplate="%{text}<br>lagged vol = %{x}<br>return = %{y}<extra></extra>"
)
fig.show()

# FIGURE 4

In [11]:
fig = px.scatter(
    x=df.vol,
    y=df.ret,
    trendline="ols",
)
fig.update_layout(
    xaxis_title=f"{name} Volatility",
    yaxis_title=f"{name} Return",
    xaxis_tickformat=".0%",
    yaxis_tickformat=".0%",
    template="plotly_white",
)
fig.update_traces(
    marker=dict(
        size=12, 
        line=dict(width=2, color="DarkSlateGrey")
    ),
    selector=dict(mode="markers"),
    text=df.index.to_list(),
    hovertemplate="%{text}<br>vol = %{x}<br>return = %{y}<extra></extra>"
)
fig.show()